In [1]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 9.7 MB/s 


In [2]:
import ee
from unidecode import unidecode
import os


In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lGonl7R3mQns23JeR8Kj1CFw3A_G3DLOeTQywK5dpW8&tc=2kXVVKwJ_Re2IOEMYHAtrL5qhHRVm9gX-3ECx60ShXw&cc=FKF64aKlD3uiGltb1JBGwYiFK0YrCNrHkGnZgw4hVOQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJp_EZjq2ftTOi4QyRuFORXnp3t8xfs0DudsknoF-cTHpZs-hh96tgA

Successfully saved authorization token.


In [6]:
gdrive_folder = 'surface_reflectance_data'


In [5]:
italy = 'users/basanthqo/Italy_prov'
county_region = ee.FeatureCollection(italy)
feature_list = county_region.toList(1e5)
feature_list_computed = feature_list.getInfo()


In [9]:
CLEAN_NAME = lambda r, l: r.get('properties').get(l)

In [8]:
def ftr_key_fn(region):
 return f"DEN_PROV_{CLEAN_NAME(region, 'DEN_PROV')}-COD_REG_{CLEAN_NAME(region,'COD_REG')}-COD_PROV_{CLEAN_NAME(region, 'COD_PROV')}"


In [7]:
# image collection bands
surf_refl_bands = ['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03','sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06','sur_refl_b07']


In [10]:
# Import the MODIA surface reflcetance image.
surf_refl = ee.ImageCollection("MODIS/061/MOD09A1")


In [ ]:
bands = surf_refl_bands

In [ ]:
def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select(bands)
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

In [ ]:
def export_to_cloud(img, fname, folder, expregion, eeuser=None, scale=500):
  # print "export to cloud"
  expcoord = expregion.geometry().coordinates().getInfo()[0]
  expconfig = dict(description=fname, folder=folder, fileNamePrefix=fname, dimensions=None, region=expcoord,
                   scale=scale, crs='EPSG:4326', crsTransform=None, maxPixels=1e13)
  task = ee.batch.Export.image.toDrive(image=img.clip(expregion), **expconfig)
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    time.sleep(10)
  print ('Done.', task.status())

In [ ]:
def image_collection(start_date,end_date,image_type,image_collection):
  imgcoll = ee.ImageCollection(image_collection).filterDate(start_date,end_date)
  img=imgcoll.iterate(appendBand)
  img=ee.Image(img)
  

In [ ]:
st_date = ['2012-01-1', '2012-01-1', '2015-01-01']
end_date = ['2022-09-30', '2022-09-30', '2022-09-30']
# bands = surf_refl_bands,surf_temp_ems_band,agri_lc_band

In [ ]:
start_date ='2012-01-1'
end_date = '2022-09-30'

In [ ]:
imgcoll = ee.ImageCollection(surf_refl).filterDate(start_date,end_date)
img=imgcoll.iterate(appendBand)
img=ee.Image(img)

In [ ]:
keys_with_issues = []
count_already_downloaded = 0
count_filtered = 0

for idx, region in enumerate(feature_list_computed):
  if not True:
      count_filtered += 1
      continue
      
  subunit_key = ftr_key_fn(region)
  file_name = f'Italy_surf_refl_{subunit_key}_{start_date}_{end_date}'
  print(file_name)
  if os.path.isfile(os.path.join(file_name + '.tif')):
          print (subunit_key, 'already downloaded. Continuing...')
          count_already_downloaded += 1
          continue

  try:
      export_to_cloud(img, file_name, gdrive_folder, ee.Feature(region), scale=500)
  except KeyboardInterrupt:
      print ('received SIGINT, terminating execution')
      break
  except Exception as e:
    print ('issue with {} ({})'.format(subunit_key, str(e)))